In [1]:
from functions_audio_model_tiny import *
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os
import json
import tensorflow as tf
import shutil
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, Mean
from sklearn.model_selection import ParameterGrid

2024-05-23 21:24:15.975134: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


First steps include proper sorting of the saved embeddings into the training, test and validation sets.

For Female sets:

In [62]:
#IDS for training set female
file_path = 'data/F_train_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_train = list(data.keys())

In [26]:
print(len(ids_train))

141


In [35]:
# Source and destination directories
src_dir = 'data/audio/embeddingsF/'
dst_dir = 'data/audio/embeddingsF_sets/F_train_set'


for filename in os.listdir(src_dir):
    if filename.endswith('.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_train:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [178]:
#count that 141 files have been moved to the training set

dir_path = 'data/audio/embeddingsF_sets/F_train_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 141


141


In [71]:
#IDS for test set female
file_path = 'data/F_test_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_test = list(data.keys())

In [48]:
len(ids_test)

30

In [43]:
# Source and destination directories
src_dir = 'data/audio/embeddingsF/'
dst_dir = 'data/audio/embeddingsF_sets/F_test_set'


for filename in os.listdir(src_dir):
    if filename.endswith('.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_test:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [179]:
#count that 30 files have been moved to the training set

dir_path = 'data/audio/embeddingsF_sets/F_test_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 30


30


In [49]:
#IDS for validation set female
file_path = 'data/F_val_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_val = list(data.keys())

In [50]:
len(ids_val)

32

In [51]:
# Source and destination directories
src_dir = 'data/audio/embeddingsF/'
dst_dir = 'data/audio/embeddingsF_sets/F_val_set'


for filename in os.listdir(src_dir):
    if filename.endswith('.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_val:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [180]:
#count that 32 files have been moved to the training set

dir_path = 'data/audio/embeddingsF_sets/F_val_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 32


32


Same procedure for the male set, this will however cause more difficulties due to the augmentations performed:

In [117]:
#IDS for training set male
file_path = 'data/M_train_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_train_M = list(data.keys())

In [118]:
len(ids_train_M)

312

In [119]:
print(ids_train_M)

['1_10462', '1_7089', '1_2819', '1_7575', '1_6065', '1_4031', '1_11913', '2_475', '1_2354', '2_541', '2_151', '1_536', '1_1638', '1_11924', '2_287', '1_1180', '1_5627', '2_128', '1_3287', '2_161', '2_124', '1_105', '2_170', '2_120', '2_392', '1_3707', '2_197', '2_373', '2_494', '2_556', '1_11378', '2_596', '2_152', '2_248', '2_523', '2_450', '2_456', '2_583', '2_288', '1_2614', '2_280', '2_247', '2_598', '1_5058', '1_8045', '1_3125', '2_550', '2_370', '1_6428', '2_39', '2_236', '1_6370', '1_7047', '1_6683', '1_7281', '1_1772', '1_1296', '2_114', '2_586', '2_242', '2_532', '1_6426', '2_103', '2_567', '2_258', '1_5964', '1_1478', '2_313', '2_359', '1_6504', '2_457', '2_377', '1_971', '1_6134', '2_174', '2_406', '2_223', '2_204', '2_474', '1_12202', '2_227', '2_449', '1_5617', '2_581', '1_1466', '1_4967', '2_509', '2_372', '1_2778', '2_433', '2_434', '1_6165', '2_115', '2_490', '1_276', '2_43', '1_6648', '2_73', '2_169', '2_511', '2_178', '1_3293', '2_622', '1_8407', '1_4145', '2_617', '1

In [115]:

# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_train_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_train_M:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [120]:
#count that 141 files have been moved to the training set

dir_path = 'data/audio/embeddingsM_sets/M_train_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 312


300


AssertionError: 

As augmentation has been done differently, the difference of 12 files is chosen at random from the Augmented Male files:

In [121]:
import random
# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_train_set'

# Get a list of all files that end with 'w2V2.npy' and their ids are in ids_train_M
files = [f for f in os.listdir(src_dir) if f.startswith('A') and f.endswith('w2V2.npy')]

# Randomly select 12 files
selected_files = random.sample(files, 12)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [126]:
#count that 312 files have been moved to the training set

dir_path = 'data/audio/embeddingsM_sets/M_train_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 312
# Print the files that start with 'A'
files_starting_with_A = [f for f in files if f.startswith('A')]

print(files_starting_with_A)
print(len(files_starting_with_A))

312
['A_2_20_w2V2.npy', 'A_2_387_w2V2.npy', 'A_1_6627_w2V2.npy', 'A_2_326_w2V2.npy', 'A_1_10190_w2V2.npy', 'A_2_208_w2V2.npy', 'A_1_8217_w2V2.npy', 'A_2_49_w2V2.npy', 'A_1_3840_w2V2.npy', 'A_1_3837_w2V2.npy', 'A_1_3333_w2V2.npy', 'A_1_3476_w2V2.npy']
12


In [127]:
#IDS for training set male
file_path = 'data/M_test_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_test_M = list(data.keys())

In [128]:
len(ids_test_M)

68

In [129]:

# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_test_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_test_M:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [ ]:
#count that 68 files have been moved to the training set

dir_path = 'data/audio/embeddingsM_sets/M_test_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 68


In [137]:

# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_test_set'
train_dir = 'data/audio/embeddingsM_sets/M_train_set'

# Get a list of all files that end with 'w2V2.npy' and their ids start with A and are not the same as the ones in the training set
files = [f for f in os.listdir(src_dir) if f.startswith('A') and f.endswith('w2V2.npy') and not os.path.exists(os.path.join(train_dir, f))]


# Randomly select 2 files
selected_files = random.sample(files, 2)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [181]:
#count that 68 files have been moved to the test set

dir_path = 'data/audio/embeddingsM_sets/M_test_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 68


68


In [146]:
#IDS for validation set male
file_path = 'data/M_val_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_val_M = list(data.keys())

In [147]:
len(ids_val_M)

68

In [149]:
print(ids_val_M)

['1_1560', '1_9971', '1_2797', '2_421', '1_5679', '1_1549', '2_189', '1_1003', '2_546', '1_10829', '1_182', '2_71', '2_478', '2_168', '2_206', '2_524', '2_155', '1_7357', '1_2580', '2_498', '1_5571', '1_11699', '2_461', '1_3064', '2_141', '1_10890', '2_464', '1_9087', '2_589', '2_148', '2_416', '1_2853', '2_491', '1_1144', 'A2_49', '1_11306', '1_2216', 'A2_261', '2_127', '2_585', '1_6113', '2_343', '2_446', '2_231', '1_1001', '2_30', '1_80', '2_536', '2_322', '2_208', '2_131', '1_7661', '1_10857', '2_86', '1_2420', '1_4792', '1_2837', '2_397', '1_5953', '2_230', '2_388', '1_6627', '2_488', '1_7490', '1_9245', 'A1_2830', '2_125', 'A1_10004']


In [143]:
# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_val_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_val_M:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [150]:
#count that 68 files have been moved to the validation set

dir_path = 'data/audio/embeddingsM_sets/M_val_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 68


64


AssertionError: 

In [151]:

# Source and destination directories
src_dir = 'data/audio/embeddingsM/'
dst_dir = 'data/audio/embeddingsM_sets/M_val_set'
train_dir = 'data/audio/embeddingsM_sets/M_train_set'
test_dir = 'data/audio/embeddingsM_sets/M_test_set'

# Get a list of all files that end with 'w2V2.npy' and their ids start with A and are not the same as the ones in the training set
files = [f for f in os.listdir(src_dir) if f.startswith('A') and f.endswith('w2V2.npy') 
         and not os.path.exists(os.path.join(train_dir, f)) and not os.path.exists(os.path.join(test_dir, f))]


# Randomly select 4 files
selected_files = random.sample(files, 4)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [182]:
#count that 68 files have been moved to the validation set

dir_path = 'data/audio/embeddingsM_sets/M_val_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 68


68


Moving on to the mixed datasets:

In [154]:
#IDS for training set mixed
file_path = 'data/mixed_train_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_train_mixed = list(data.keys())

In [155]:
len(ids_train_mixed)

600

In [157]:
# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_train_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_train_mixed:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [160]:
#count that 68 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_train_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 600


440


AssertionError: 

In [161]:

# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_train_set'

# Get a list of all files that end with 'w2V2.npy' and their ids start with A and are not the same as the ones in the training set
files = [f for f in os.listdir(src_dir) if (f.startswith('A1') or f.startswith('A2')) and f.endswith('w2V2.npy')]

# Randomly select 160 files
selected_files = random.sample(files, 160)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [162]:
#count that 600 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_train_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 600


600


In [163]:
#IDS for training set mixed
file_path = 'data/mixed_test_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_test_mixed = list(data.keys())

In [164]:
len(ids_test_mixed)

130

In [165]:
# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_test_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_test_mixed:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [167]:
#count that 130 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_test_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 130


98


AssertionError: 

In [168]:

# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_test_set'
train_dir = 'data/audio/embeddingsmixed_sets/mixed_train_set'

# Get a list of all files that end with 'w2V2.npy' and their ids start with A and are not the same as the ones in the training set
files = [f for f in os.listdir(src_dir) if (f.startswith('A1') or f.startswith('A2')) and f.endswith('w2V2.npy')
         and not os.path.exists(os.path.join(train_dir, f))]

# Randomly select 32 files
selected_files = random.sample(files, 32)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [169]:
#count that 130 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_test_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 130


130


In [170]:
#IDS for validation set mixed
file_path = 'data/mixed_val_set.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Extract the IDs
ids_val_mixed = list(data.keys())

In [171]:
len(ids_val_mixed)

130

In [172]:
# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_val_set'


for filename in os.listdir(src_dir):
    if filename.endswith('w2V2.npy'):
        # Extract the id from the filename
        id = filename.split('_w2V2.')[0]
        
        if id in ids_val_mixed:
            source_file = os.path.join(src_dir, filename)
            target_file = os.path.join(dst_dir, filename)
            
            shutil.copy(source_file, target_file)

In [175]:
#count that 130 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_val_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 130


95


AssertionError: 

In [176]:
# Source and destination directories
src_dir = 'data/audio/embeddingsmixed/'
dst_dir = 'data/audio/embeddingsmixed_sets/mixed_val_set'
train_dir = 'data/audio/embeddingsmixed_sets/mixed_train_set'
test_dir = 'data/audio/embeddingsmixed_sets/mixed_test_set'

# Get a list of all files that end with 'w2V2.npy' and their ids start with A and are not the same as the ones in the training set
files = [f for f in os.listdir(src_dir) if (f.startswith('A1') or f.startswith('A2')) and f.endswith('w2V2.npy')
         and not os.path.exists(os.path.join(train_dir, f)) and not os.path.exists(os.path.join(test_dir, f))]


# Randomly select 35 files
selected_files = random.sample(files, 35)

# Copy the selected files to the destination directory
for filename in selected_files:
    source_file = os.path.join(src_dir, filename)
    target_file = os.path.join(dst_dir, filename)
    
    shutil.copy(source_file, target_file)

In [177]:
#count that 130 files have been moved to the validation set

dir_path = 'data/audio/embeddingsmixed_sets/mixed_val_set'

files = os.listdir(dir_path)

print(len(files))
assert len(files) == 130


130


Now to the creation of the correct labels for the correct files:

In [6]:
#Train set labels extraction

with open('data/audio/labels_F.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsF_sets/F_train_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_F_train = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsF_sets/labels_F_train.json', 'w') as f:
    json.dump(labels_F_train, f)

In [5]:
print(len(labels_F_train))

141


In [7]:
#Test set labels extraction

with open('data/audio/labels_F.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsF_sets/F_test_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_F_test = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsF_sets/labels_F_test.json', 'w') as f:
    json.dump(labels_F_test, f)

In [15]:
print(len(labels_F_test))

30


In [8]:
#Val set labels extraction

with open('data/audio/labels_F.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsF_sets/F_val_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_F_val = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsF_sets/labels_F_val.json', 'w') as f:
    json.dump(labels_F_val, f)

In [16]:
print(len(labels_F_val))

32


In [9]:
#Train set labels extraction

with open('data/audio/labels_M.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsM_sets/M_train_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_M_train = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsM_sets/labels_M_train.json', 'w') as f:
    json.dump(labels_M_train, f)

In [17]:
print(len(labels_M_train))

312


In [12]:
#Test set labels extraction

with open('data/audio/labels_M.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsM_sets/M_test_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_M_test = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsM_sets/labels_M_test.json', 'w') as f:
    json.dump(labels_M_test, f)

In [19]:
print(len(labels_M_test))

68


In [13]:
#Val set labels extraction

with open('data/audio/labels_M.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsM_sets/M_val_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_M_val = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsM_sets/labels_M_val.json', 'w') as f:
    json.dump(labels_M_val, f)

In [20]:
print(len(labels_M_val))

68


In [14]:
#Train set labels extraction

with open('data/audio/labels_mixed.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsmixed_sets/mixed_train_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_mixed_train = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsmixed_sets/labels_mixed_train.json', 'w') as f:
    json.dump(labels_mixed_train, f)

In [21]:
print(len(labels_mixed_train))

600


In [22]:
#Test set labels extraction

with open('data/audio/labels_mixed.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsmixed_sets/mixed_test_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_mixed_test = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsmixed_sets/labels_mixed_test.json', 'w') as f:
    json.dump(labels_mixed_test, f)

In [24]:
print(len(labels_mixed_test))

130


In [23]:
#Val set labels extraction

with open('data/audio/labels_mixed.json', 'r') as f:
    data = json.load(f)

files = [f for f in os.listdir('data/audio/embeddingsmixed_sets/mixed_val_set') if f.endswith('w2V2.npy')]

#Extract the ids from the filenames
ids = [f.split('_w2V2.')[0] for f in files]

#Extract only the labels that match the ids
labels_mixed_val = {id: data[id] for id in data if id in ids}

# Save the extracted labels to a new JSON file
with open('data/audio/embeddingsmixed_sets/labels_mixed_val.json', 'w') as f:
    json.dump(labels_mixed_val, f)

In [25]:
print(len(labels_mixed_val))

130
